In [32]:
#5a
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np 

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split

s3 = boto3.resource('s3')
bucket_name = "rachaeld-data445"
bucket = s3.Bucket(bucket_name)

file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the data-file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [34]:
#5b 

# defining input and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

# defining the list to store estimated coefficients at each split
coef = list()

for i in range(0,1000):
    
    # splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2)
    
    # running LASSO cross validation, to estimate optimal lambda 
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)
    
    # building LASSO Regression with optimal lambda
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    # storing estimated coefficients
    coef.append(lasso_md.coef_)
    
# putting list as dataframe
coef_data = pd.DataFrame(coef)

In [35]:
coef_data.head()

,0,1,2,3,4,5,6
0,119.172814,127.172371,295.096999,45.408576,1.903427,-77.198620,-0.000000
1,150.158585,109.615777,308.039285,45.569555,1.962655,-116.682698,-0.000000
2,186.325536,120.022626,277.032214,42.068934,1.956275,-50.373678,-41.841059
3,148.447173,116.360434,263.352138,40.323407,2.295718,-112.978049,-13.031832
4,208.030463,107.713943,251.564939,49.050633,1.724134,-111.503735,0.000000


In [36]:
print(sum(coef_data[0] == 0))
print(sum(coef_data[1] == 0))
print(sum(coef_data[2] == 0))
print(sum(coef_data[3] == 0))
print(sum(coef_data[4] == 0))
print(sum(coef_data[5] == 0))
print(sum(coef_data[6] == 0))
#should drop highwaympg(6) because so many 0s

0
0
0
0
0
18
772


In [37]:
#5c
# dropping highwaympg
X_train = X_train.drop(columns = ['highwaympg'], axis = 1)
X_test = X_test.drop(columns = ['highwaympg'], axis = 1)

def l2_normalization(X):
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X-x_mean) / l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [38]:
#5d
# linear regression
lm_md = LinearRegression().fit(X_train, Y_train)

# predicting on test
lm_pred = lm_md.predict(X_test)

#computing the mse of the linear regression model
mse1 = np.mean(np.power(Y_test-lm_pred, 2))
print('The mse of the linear model is:', mse1)

The mse of the linear model is: 8594288.104014887


In [46]:
#5e
ridgeres = list()
for k in range (0,100):
    # Ridge regression
    ridge_cv = RidgeCV(alphas = [.001, .01, .1, 1, 10, 100], cv= 5).fit(X_train, Y_train)

    # extracting the best lambda
    cv_lambda = ridge_cv.alpha_
    #### from the ica print('The best lambda of the ridge model is:', cv_lambda)
    
    #storing in list
    ridgeres.append(cv_lambda)
    
import statistics 
from statistics import mode
bestlambda = mode(ridgeres)
print(bestlambda)

# building the ridge model
ridge_md = Ridge(alpha = bestlambda).fit(X_train, Y_train)

# predicting on test
ridge_pred = ridge_md.predict(X_test)

# computing the mse
mse2 = np.mean(np.power(Y_test-ridge_pred, 2))
print('The mse of the ridge model is:', mse2)

0.001
The mse of the ridge model is: 11261428.009363435
